# Injecting MFA results into LCA using Datapackage arrays

In [1]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import numpy as np

10:01:14+0000 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

In [3]:
laminate = bd.get_node(name='market for photovoltaic laminate, CIS', database="PV production")

In [4]:
for exc in laminate.technosphere():
    print(exc.input["database"], exc)

PV production Exchange: 0.639999999999999 square meter 'photovoltaic laminate production, CIS' (square meter, RoW, None) to 'market for photovoltaic laminate, CIS' (square meter, GLO, None)
PV production Exchange: 0.360000000000001 square meter 'photovoltaic laminate production, CIS' (square meter, DE, None) to 'market for photovoltaic laminate, CIS' (square meter, GLO, None)


## Get the indices of the edges to be modified

`bw_processing` Datapackage edges are defined by `product, process`, regardless of whether it is a consumption or production edge. This is because the order corresponds to `row, column`:

In [5]:
bwp.INDICES_DTYPE

[('row', numpy.int64), ('col', numpy.int64)]

In this case, we will update the two exchange values for this market. You will need to define your MFA system, and find correct node `id` values.

In [6]:
production_row = bd.get_node(
    name="photovoltaic laminate production, CIS", 
    location="RoW",
    database="ecoinvent-3.12-cutoff"
)
production_de = bd.get_node(
    name="photovoltaic laminate production, CIS", 
    location="DE",
    database="ecoinvent-3.12-cutoff"
)

In [7]:
indices = np.array(
    [
        # Product (row), process (column)
        (production_row.id, laminate.id),   
        (production_de.id, laminate.id),   
    ],
    dtype=bwp.INDICES_DTYPE
)

We already know that these are both _consumption_ edges, so we need to flip their signs when injecting them into the matrix. Consumption edges are negative, production edges are positive. We can accomplish this sign flipping with a flip array (has same lenght and order as the indices):

In [8]:
flip_array = np.array([True, True])

## Define the data

Our data array has rows of matrix elements to replace, in the _same order_ as the indices are defined.

It has columns of possible values - from scenarios, time steps, some combination of the two, or however else you defined your MFA dimensionality.

The data array doesn't include any labelling - this is something we will have to keep track of manually. Let's set up our column definitions:

In [9]:
columns = [
    ("BAU", 2030),
    ("AMB", 2030),
    ("BAU", 2040),
    ("AMB", 2040),
]

We can now pull our data from the MFA system. It's up to you how this works, but in the end you are responsible for creating an array where the data lines up with the defined columns and rows.

You don't need the following formatting, I just tried to make it explcit what lined up with what.

In [10]:
data = np.array(
    [
        # ("BAU", 2030), ("AMB", 2030), ("BAU", 2040), ("AMB", 2040)
        [  0.7,           0.6,           0.8,           0.5],  # (production_row.id, laminate.id)
        [  0.3,           0.4,           0.2,           0.5],  # (production_de.id, laminate.id),
    ]
)

## Building the datapackage

We can now put this data into a datapackage. We `sequential` so that when we iterate we get our scenarios in order.

In [11]:
dp = bwp.create_datapackage(sequential=True)

In [12]:
dp.add_persistent_array

<bound method Datapackage.add_persistent_array of <bw_processing.datapackage.Datapackage object at 0x7f245f6b15e0>>

In [13]:
dp.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=indices,
    data_array=data,
    flip_array=flip_array,
)

## Using the datapackage

To use the datapackage, we need to get our "base" datapackages for the LCA (includes technosphere and biosphere matrix definitions, but also the characterization matrix), and then create a list with those datapackages and our new datapackage:

In [14]:
random_ic = (
    'ecoinvent-3.12',
    'ReCiPe 2016 v1.03, endpoint (I) no LT',
    'total: human health no LT',
    'human health no LT'
)

In [15]:
fu, datapackages, _ = bd.prepare_lca_inputs({laminate: 1}, method=random_ic)

Let's do a first calculation to get a baseline score:

In [16]:
lca = bc.LCA(
    demand=fu,
    data_objs=datapackages,
)
lca.lci()
lca.lcia()
ref_score = lca.score * 1000
ref_score

0.06494229717246773

We can now use our new datapackage. Don't forget to pass `use_arrays=True`; otherwise our fancy new data will be ignored!

In [17]:
datapackages.append(dp)

In [18]:
lca = bc.LCA(
    demand=fu,
    data_objs=datapackages,
    use_arrays=True
)
lca.lci()
lca.lcia()
lca.score * 1000

0.13370650483310634

If we want to iterate over all the scenarios/columns, it's convenient to write a loop. However, if we call `next()` we will already advance to the next scneario. We can use `lca.keep_first_iteration()` to return the first result during our loop, and only on the next iteration move to the second scenario:

In [19]:
lca.keep_first_iteration()

We can now iterate and record our scores. You can modify this code to also do other interpretation steps with the `characterized_inventory` matrix - you could even do grah traversal on each iteration.

In [20]:
from tqdm import tqdm

In [21]:
results = []

# This looks like magic, but by putting `lca` here it will automatically call next() on the 
# LCA and do a complete calculation automatically.
for label, _ in tqdm(zip(columns, lca)):
    results.append((label, lca.score * 1000, lca.score * 1000 / ref_score))

results

4it [00:04,  1.07s/it]


[(('BAU', 2030), 0.13370650483310634, 2.0588508669168415),
 (('AMB', 2030), 0.12733665553817886, 1.960766112107337),
 (('BAU', 2040), 0.14007635412803388, 2.156935621726347),
 (('AMB', 2040), 0.12096680624325151, 1.8626813572978345)]